# Example using DAQ plus AR Parameters plus Mahalanobis Distance

This notebook demonstrates an integrated approach combining:
- Data acquisition (DAQ) systems
- Autoregressive (AR) parameter extraction
- Mahalanobis distance-based outlier detection

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shmtools.features import ARModelExtractor
from shmtools.detection import MahalanobisDetector
from shmtools.daq import simulate_daq_data

## Step 1: Simulate DAQ Data Acquisition

In [ ]:
# Simulate data acquisition from sensors
fs = 1000  # Sampling frequency (Hz)
duration = 10  # Duration (seconds)
n_channels = 4  # Number of sensor channels

# Generate simulated sensor data
time = np.linspace(0, duration, int(fs * duration))
data = np.random.randn(len(time), n_channels)

# Add some structural response characteristics
for i in range(n_channels):
    freq = 10 + i * 5  # Different frequency content per channel
    data[:, i] += 0.5 * np.sin(2 * np.pi * freq * time)

print(f"DAQ data shape: {data.shape}")
print(f"Sampling rate: {fs} Hz")
print(f"Number of channels: {n_channels}")

## Step 2: Extract AR Parameters

In [ ]:
# Initialize AR model extractor
ar_extractor = ARModelExtractor(order=10)

# Extract AR features from each channel
ar_features = []
for ch in range(n_channels):
    features = ar_extractor.extract(data[:, ch])
    ar_features.append(features)

ar_features = np.array(ar_features)
print(f"AR features shape: {ar_features.shape}")

## Step 3: Apply Mahalanobis Distance Detection

In [ ]:
# Train Mahalanobis detector on baseline features
detector = MahalanobisDetector()

# Use first half as training (baseline) data
train_features = ar_features[:n_channels//2]
test_features = ar_features[n_channels//2:]

# Fit detector on baseline
detector.fit(train_features)

# Detect outliers in test data
distances = detector.decision_function(test_features)
predictions = detector.predict(test_features)

print(f"Mahalanobis distances: {distances}")
print(f"Outlier predictions: {predictions}")

## Step 4: Visualize Results

In [ ]:
# Plot the integrated results
fig, axes = plt.subplots(3, 1, figsize=(12, 10))

# Plot raw DAQ data
axes[0].plot(time[:1000], data[:1000, 0], label='Channel 1')
axes[0].set_xlabel('Time (s)')
axes[0].set_ylabel('Amplitude')
axes[0].set_title('Raw DAQ Data Sample')
axes[0].legend()
axes[0].grid(True)

# Plot AR coefficients
axes[1].bar(range(len(ar_features[0])), ar_features[0])
axes[1].set_xlabel('AR Coefficient Index')
axes[1].set_ylabel('Value')
axes[1].set_title('AR Parameters for Channel 1')
axes[1].grid(True)

# Plot Mahalanobis distances
axes[2].bar(range(len(distances)), distances, 
           color=['red' if p == -1 else 'green' for p in predictions])
axes[2].set_xlabel('Test Sample')
axes[2].set_ylabel('Mahalanobis Distance')
axes[2].set_title('Outlier Detection Results')
axes[2].axhline(y=detector.threshold_, color='r', linestyle='--', label='Threshold')
axes[2].legend()
axes[2].grid(True)

plt.tight_layout()
plt.show()

## Conclusion

This integrated example demonstrates:
1. Data acquisition from multiple sensor channels
2. Feature extraction using AR model parameters
3. Outlier detection using Mahalanobis distance
4. Visualization of the complete pipeline

This approach is commonly used in structural health monitoring to detect anomalies or damage in structures by combining signal processing with statistical outlier detection methods.